In [ ]:
import pandas as pd

In [42]:
tweets = df[['id', 'tweet', 'date', 'retweets_count', 'replies_count', 'likes_count', 'quote_url']]

In [44]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 0 to 97
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              178 non-null    int64 
 1   tweet           178 non-null    object
 2   date            178 non-null    object
 3   retweets_count  178 non-null    int64 
 4   replies_count   178 non-null    int64 
 5   likes_count     178 non-null    int64 
 6   quote_url       9 non-null      object
dtypes: int64(4), object(3)
memory usage: 11.1+ KB


In [45]:
users = df[['user_id', 'username']]

In [46]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178 entries, 0 to 97
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   178 non-null    int64 
 1   username  178 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.2+ KB


In [ ]:
from textblob import TextBlob
from IPython.display import Markdown, display
# Pretty printing the result
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))
for tweet in tweets:
    print(tweet)
    analysis = TextBlob(tweet)
    print(analysis.sentiment)
    if analysis.sentiment[0]>0:
        printmd('Positive', color="green")
    elif analysis.sentiment[0]<0:
        printmd('Negative', color="red")
    else:
        printmd("No result", color="grey")
        print("")

## crear db vacía

In [49]:
import pymysql

In [50]:
import sqlite3

In [51]:
connection = sqlite3.connect('thebridge_tweets.db')

In [52]:
cursor = connection.cursor()

## crear las dos tablas, y pasarles la info de los anteriores dfs 

In [53]:
cursor.execute('''CREATE TABLE IF NOT EXISTS tweets_table
              (indx INT SERIAL NOT NULL, id INT, tweet TEXT, date TEXT, retweets_count INT, replies_count INT, likes_count INT, quote_url TEXT, primary key (indx))''')

In [54]:
tweets.reset_index(drop=True, inplace=True)

In [55]:
tweets.to_sql('tweets_table', connection, if_exists='replace', index = False)

In [56]:
cursor.execute('''CREATE TABLE IF NOT EXISTS users_table
              (indx INT SERIAL NOT NULL, user_id INT, username TEXT, primary key (indx))''')

In [57]:
users.reset_index(drop=True, inplace=True)

In [58]:
users.to_sql('users_table', connection, if_exists='replace', index = False)

## comprobar

In [59]:
res = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

tweets_table
users_table


In [60]:
cursor.execute("SELECT * from tweets_table").fetchall()

[(1577654769098842112,
  'Ya están aquí nuestros primeros mini emprendedores 🤩 pasándoselo en grande mientras empiezan el proceso de ideación de sus proyectos-   Con el programa de @GenyusSchool descubren su YO emprendedor mientras se divierten👏  ¡Aún estás a tiempo!👉 https://t.co/uaEMCO1jGy  https://t.co/80gibJEUcI',
  '2022-10-05',
  2,
  0,
  3,
  None),
 (1577564440530124802,
  '¡Feliz Día a nuestros Lead Instructors y Teacher Assistants! 🎉 Sabemos que en ocasiones un #bootcamp es duro para tod@s pero cada día dais el 💯%  ¡Gracias por hacer lo #digital mucho más fácil!  #diamundialdelosdocentes #data #ciber #desarrollowebfullstack #mkt #uxui #cloud  https://t.co/Qnehy8ODCq',
  '2022-10-05',
  2,
  0,
  2,
  None),
 (1577334577701453827,
  'Recordaros que la semana que viene tenemos la primera quedada #CPS en Madrid, en @TheBridge_Tech, en el que tendremos la gran suerte de contar con @Recuenco para que nos dé una charla. Las plazas se han agotado, pero os podéis apuntar al grupo pa

## pequeño análisis: 6 cuestiones

1. ¿Cuál es el tweet con mayor repercusión social?

In [61]:
tweets.head()

,id,tweet,date,retweets_count,replies_count,likes_count,quote_url
0,1577654769098842112,Ya están aquí nuestros primeros mini emprended...,2022-10-05,2,0,3,NaN
1,1577564440530124802,¡Feliz Día a nuestros Lead Instructors y Teach...,2022-10-05,2,0,2,NaN
2,1577334577701453827,Recordaros que la semana que viene tenemos la ...,2022-10-04,2,0,12,NaN
3,1576951801047179264,¡Vuelven las #becas Digital Talent Pipeline! ...,2022-10-03,1,0,1,NaN
4,1576902991507922944,El desarrollador web es uno de los perfiles má...,2022-10-03,3,0,3,NaN


In [62]:
tweets['repercusion'] = tweets['retweets_count'] + tweets['replies_count'] + tweets['likes_count'] 

c:\Users\IRENE\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
tweets[tweets['repercusion'] == tweets['repercusion'].max()]

,id,tweet,date,retweets_count,replies_count,likes_count,quote_url,repercusion
159,1541046264539893760,"🕣 TIC, TAC: MAÑANA os vemos a todos a las 7.30...",2022-06-26,6,6,17,https://twitter.com/HeavyMental_es/status/1535...,29


In [64]:
tweets['tweet'][159]

'🕣 TIC, TAC: MAÑANA os vemos a todos a las 7.30pm en nuestro evento en directo en MADRID en @TheBridge_Tech   🚨Para los que no vayáis, TRANQUILOS, seguramente el martes lo emitamos para todos vosotros :D'

2. ¿Cuál es el usuario que más menciona a la escuela?

In [73]:
df['username'].value_counts()

thebridge_tech     32
heavymental_es     12
mulder01198676      9
joobid              6
designetworking     6
                   ..
haycanal            1
ceeivalencia        1
carlosmcabo         1
_miki               1
pcontrer            1
Name: username, Length: 80, dtype: int64

In [66]:
df['username'].nunique()

80

In [76]:
df['username'].value_counts().max() / len(df)

0.1797752808988764

3. ¿En qué mes se concentra el mayor número de tweets?

In [105]:
df['date'].head()

0    2022-10-05
1    2022-10-05
2    2022-10-04
3    2022-10-03
4    2022-10-03
Name: date, dtype: object

In [85]:
df[["day", "month", "year"]] = df["date"].str.split("-", expand = True)

In [86]:
df['month'].value_counts()

06    93
09    32
07    28
08    19
10     6
Name: month, dtype: int64

4. ¿Qué palabras son más frecuentes?

In [87]:
text = tweets['tweet']

In [88]:
len(text)

178

In [89]:
for i in range(40): 
    print('####################')
    print(text[i])

####################
Ya están aquí nuestros primeros mini emprendedores 🤩 pasándoselo en grande mientras empiezan el proceso de ideación de sus proyectos-   Con el programa de @GenyusSchool descubren su YO emprendedor mientras se divierten👏  ¡Aún estás a tiempo!👉 https://t.co/uaEMCO1jGy  https://t.co/80gibJEUcI
####################
¡Feliz Día a nuestros Lead Instructors y Teacher Assistants! 🎉 Sabemos que en ocasiones un #bootcamp es duro para tod@s pero cada día dais el 💯%  ¡Gracias por hacer lo #digital mucho más fácil!  #diamundialdelosdocentes #data #ciber #desarrollowebfullstack #mkt #uxui #cloud  https://t.co/Qnehy8ODCq
####################
Recordaros que la semana que viene tenemos la primera quedada #CPS en Madrid, en @TheBridge_Tech, en el que tendremos la gran suerte de contar con @Recuenco para que nos dé una charla. Las plazas se han agotado, pero os podéis apuntar al grupo para próximas quedadas 🚀
####################
¡Vuelven las #becas Digital Talent Pipeline!  1⃣0⃣0⃣ be

In [90]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\¡)|(\?)|(\¿)|(\@)|(\#)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_text(text):
    
    text = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in text]
    text = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in text]
    
    return text

# text tras aplicar la limpieza
text_clean = preprocess_text(text)


In [91]:
text_clean

['ya están aquí nuestros primeros mini emprendedores 🤩 pasándoselo en grande mientras empiezan el proceso de ideación de sus proyectos    con el programa de genyusschool descubren su yo emprendedor mientras se divierten👏  aún estás a tiempo👉 https  tco uaemcojgy  https  tco gibjeuci',
 'feliz día a nuestros lead instructors y teacher assistants 🎉 sabemos que en ocasiones un bootcamp es duro para tods pero cada día dais el 💯%  gracias por hacer lo digital mucho más fácil  diamundialdelosdocentes data ciber desarrollowebfullstack mkt uxui cloud  https  tco qnehyodcq',
 'recordaros que la semana que viene tenemos la primera quedada cps en madrid en thebridge_tech en el que tendremos la gran suerte de contar con recuenco para que nos dé una charla las plazas se han agotado pero os podéis apuntar al grupo para próximas quedadas 🚀',
 'vuelven las becas digital talent pipeline  ⃣⃣⃣ becas para formarte en la próxima edición de nuestros bootcamp en madrid valencia y sevilla  solicita ya tu beca

In [92]:
from nltk.corpus import stopwords

# Aplicamos la eliminacion de las palabras directamente sobre las reviews
# Demasiado manual. Mejor sobre el CountVectorizer (ver abajo)
spanish_stop_words = stopwords.words('spanish')

def remove_stop_words(text):
    removed_stop_words = []
    for review in text:
        
        # Para cada review elimina las stopwords, y separa todas las palabras por espacio si no es stopword
        removed_stop_words.append(
            ' '.join([word for word in review.split() if word not in spanish_stop_words])
        )
        
    return removed_stop_words

# Se lo aplicamos antes de vectorizar
no_stop_words_text = remove_stop_words(text_clean)

In [93]:
from sklearn.feature_extraction.text import CountVectorizer 

In [94]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(no_stop_words_text)
#vectorizer.get_feature_names()

In [95]:
baseline_vectorizer = CountVectorizer(binary=True)
baseline_vectorizer.fit(no_stop_words_text)

CountVectorizer(binary=True)

In [96]:
X_baseline = baseline_vectorizer.transform(no_stop_words_text)


In [97]:
x = baseline_vectorizer.vocabulary_

In [103]:
sorted_x = sorted(x.items(), key=lambda kv: kv[1], reverse=True)
top_words = sorted_x[:50]
top_words[:25]

[('𝘁𝗿𝗶𝗽𝘂𝗹𝗮𝗰𝗶𝗼𝗻𝗲𝘀', 1344),
 ('𝗱𝗲', 1343),
 ('𝗗𝗲𝘀𝗮𝗳𝗶𝗼', 1342),
 ('último', 1341),
 ('últimas', 1340),
 ('última', 1339),
 ('éxito', 1338),
 ('área', 1337),
 ('ánimo', 1336),
 ('ámbito', 1335),
 ('ágora', 1334),
 ('zvmtzttnkm', 1333),
 ('zuptdzn', 1332),
 ('zuoqnwwch', 1331),
 ('zoom', 1330),
 ('zonas', 1329),
 ('zhovlnbp', 1328),
 ('zgvkix', 1327),
 ('zas', 1326),
 ('yvwlopivv', 1325),
 ('ytrvxkq', 1324),
 ('ytrvao', 1323),
 ('ytpokcith', 1322),
 ('youtube', 1321),
 ('you', 1320)]

In [104]:
top_words[25:]


[('yotethhikx', 1319),
 ('ylrazyt', 1318),
 ('yesterday', 1317),
 ('yes', 1316),
 ('yellowers', 1315),
 ('ycxvjjbf', 1314),
 ('yckmxvai', 1313),
 ('ya', 1312),
 ('xpaazxxsdx', 1311),
 ('xnwjkufpi', 1310),
 ('xmywvhvvl', 1309),
 ('xdd', 1308),
 ('wzrbwvck', 1307),
 ('wpfmnlmd', 1306),
 ('working', 1305),
 ('with', 1304),
 ('were', 1303),
 ('weolgquxm', 1302),
 ('wellness', 1301),
 ('week', 1300),
 ('webapps', 1299),
 ('web', 1298),
 ('we', 1297),
 ('wbscodingschool', 1296),
 ('was', 1295)]

5. ¿Qué tipo de correlación matemática encuentras entre las métricas públicas?

In [ ]:
# lineal directa fuerte entre likes y retweets, más débil entre likes y replies

In [99]:
metrics = tweets[['retweets_count', 'replies_count', 'likes_count']]
metrics.corr()

,retweets_count,replies_count,likes_count
retweets_count,1.000000,0.136691,0.670217
replies_count,0.136691,1.000000,0.311805
likes_count,0.670217,0.311805,1.000000


In [100]:
import phik

In [101]:
metrics.phik_matrix()

interval columns not set, guessing: ['retweets_count', 'replies_count', 'likes_count']


,retweets_count,replies_count,likes_count
retweets_count,1.000000,0.290227,0.732532
replies_count,0.290227,1.000000,0.443209
likes_count,0.732532,0.443209,1.000000


6. ¿Has sacado alguna conclusión extra en tu análisis?

- el Desafío de Trpulaciones es el tema que más impulsa la publicación de tweets
- la relación entre tweets y días lectivos es mayor en agosto que en septiembre
- el espacio más popular de la escuela es el ágora
- la escuela que más se menciona es wbscodingschool
- la plataforma más mencionada es youtube
